<div align=center><img src="1.png"></div>

<div align=center><img src="2.png"></div>

<div align = 'center'>
    <font size = '20'><b>本科实验报告</b></font>
</div>

<div align = 'center'>

|   姓名：   |       沈骏一       |
| :--------: | :----------------: |
|   学院：   | 控制科学与工程学院 |
|   专业：   |   自动化（控制）   |
|   学号：   |     3200100259     |
| 指导教师： |        姜伟        |
</div>

<div align = 'center'>
    <font size = '5'><b>2023年5月9日</b></font>
</div>
 <div align = 'center'>
    <font size = '5'><b> DIP 图像复原</b></font>
</div>

1、任选一张清晰图像，模仿课件代码添加移动模糊或失焦模糊；

2、实现逆滤波和其它复原方法（比如维纳滤波、有约束复原方法等），利用正确的PSF和不正确的PSF对模糊图像复原，展示复原结果；

3、对步骤1移动模糊图像添加不同水平的噪声，重复步骤2工作；

4、提交课程报告，内容包括各步骤代码和实现结果，并讨论各方法的优缺点。

## 添加模糊

In [237]:
import cv2
image = cv2.imread('image.png')
print(image.shape)
cv2.imshow('Original Image', image)
cv2.waitKey()

(900, 1600, 3)


-1

In [234]:
import numpy as np
def mul(A, B):
    ra, ca = A.shape
    rb, cb = B.shape
    r = ra + rb - 1
    c = ca + cb - 1

    a1 = np.pad(A, ((0, r - ra), (0, c - ca)), 'constant')
    b1 = np.pad(B, ((0, r - rb), (0, c - cb)), 'constant')
    fftA = np.fft.fft2(a1)
    fftB = np.fft.fft2(b1)
    result1 = np.fft.ifft2(fftA * fftB)

    r_start = int(np.floor(rb / 2)) + 1
    r_end = r_start + ra - 1
    c_start = int(np.floor(cb / 2)) + 1
    c_end = c_start + ca - 1
    out = result1[r_start:r_end+1, c_start:c_end+1]
    return out

def div(A, B):
    ra, ca = A.shape
    rb, cb = B.shape
    r = ra + rb - 1
    c = ca + cb - 1

    a1 = np.pad(A, ((0, r - ra), (0, c - ca)), 'constant')
    b1 = np.pad(B, ((0, r - rb), (0, c - cb)), 'constant')
    fftA = np.fft.fft2(a1)
    fftB = np.fft.fft2(b1)+1e-8
    result1 = np.fft.ifft2(fftA / fftB)

    r_start = int(np.floor(rb / 2)) + 1
    r_end = r_start + ra - 1
    c_start = int(np.floor(cb / 2)) + 1
    c_end = c_start + ca - 1
    out = result1[r_start:r_end+1, c_start:c_end+1]
    return out

In [231]:
def motion_blur(image, kernel_size=15):
    kernel_motion_blur = np.zeros((kernel_size, kernel_size))
    kernel_motion_blur[int((kernel_size-1)/2), :] = np.ones(kernel_size)
    cv2.normalize(src= kernel_motion_blur, dst= kernel_motion_blur, norm_type = cv2.NORM_L1)
    #blurred = cv2.filter2D(image, -1, kernel_motion_blur)
    blurred = np.zeros_like(image)
    f_kernel_motion_blur = np.fft.fft2(kernel_motion_blur,s=(image.shape[0],image.shape[1]))
    for i in range(3):
        f_image = np.fft.fft2(image[:,:,i])
        f_image = f_image * f_kernel_motion_blur
        f_image = np.fft.ifft2(f_image)
        f_image = np.abs(f_image)
        blurred[:,:,i] = f_image / f_image.max() * 255

    
    return blurred,kernel_motion_blur,f_kernel_motion_blur

motion_blurred_image,kernel_motion_blur,f_kernel_motion_blur = motion_blur(image)
cv2.imshow('Motion Blurred Image', motion_blurred_image)
cv2.waitKey()


-1

In [238]:
def motion_blur(image, kernel_size=15):
    kernel_motion_blur = np.zeros((kernel_size, kernel_size))
    kernel_motion_blur[int((kernel_size-1)/2), :] = np.ones(kernel_size)
    cv2.normalize(src= kernel_motion_blur, dst= kernel_motion_blur, norm_type = cv2.NORM_L1)
    #blurred = cv2.filter2D(image, -1, kernel_motion_blur)
    blurred = np.zeros_like(image)
    for i in range(3):
        blurred[:,:,i] = mul(image[:,:,i],kernel_motion_blur)

    
    return blurred,kernel_motion_blur,f_kernel_motion_blur

motion_blurred_image,kernel_motion_blur,f_kernel_motion_blur = motion_blur(image)
cv2.imshow('Motion Blurred Image', motion_blurred_image)
cv2.waitKey()

C:\Users\10037\AppData\Local\Temp\ipykernel_22988\3891836310.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  blurred[:,:,i] = mul(image[:,:,i],kernel_motion_blur)


-1

In [205]:
def defocus_blur(image, kernel_size=15):
    kernel_defocus_blur = np.ones((kernel_size, kernel_size))
    cv2.normalize(src= kernel_defocus_blur, dst= kernel_defocus_blur, norm_type = cv2.NORM_L1)
    #blurred = cv2.filter2D(image, -1, kernel_defocus_blur)
    blurred = np.zeros_like(image)
    f_kernel_defocus_blur = np.fft.fft2(kernel_defocus_blur,s=(image.shape[0],image.shape[1]))
    for i in range(3):
        f_image = np.fft.fft2(image[:,:,i])
        f_image = f_image * f_kernel_defocus_blur
        f_image = np.fft.ifft2(f_image)
        f_image = np.abs(f_image)
        blurred[:,:,i] = f_image / f_image.max() * 255
        

    return blurred,kernel_defocus_blur

defocus_blurred_image,kernel_defocus_blur = defocus_blur(image)
cv2.imshow('Defocus Blurred Image', defocus_blurred_image)
cv2.waitKey()

-1

In [235]:
def defocus_blur(image, kernel_size=15):
    kernel_defocus_blur = np.ones((kernel_size, kernel_size))
    cv2.normalize(src= kernel_defocus_blur, dst= kernel_defocus_blur, norm_type = cv2.NORM_L1)
    #blurred = cv2.filter2D(image, -1, kernel_motion_blur)
    blurred = np.zeros_like(image)
    for i in range(3):
        blurred[:,:,i] = mul(image[:,:,i],kernel_defocus_blur)

    
    return blurred,kernel_defocus_blur

defocus_blurred_image,kernel_defocus_blur = defocus_blur(image)
cv2.imshow('Defocus Blurred Image', defocus_blurred_image)
cv2.waitKey()

C:\Users\10037\AppData\Local\Temp\ipykernel_22988\2110375411.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  blurred[:,:,i] = mul(image[:,:,i],kernel_defocus_blur)


-1

In [228]:
def inverse_filter(image, psf):
    inversed = np.zeros_like(image)
    psf_fft = np.fft.fft2(psf,s=(image.shape[0],image.shape[1]))+1e-6
    for i in range(3):
        f_image = np.fft.fft2(image[:,:,i])
        f_image = f_image / psf_fft
        f_image = np.fft.ifft2(f_image)
        f_image = np.abs(f_image)
        inversed[:,:,i] = f_image / f_image.max() * 255
    return inversed.astype(np.uint8)

inverse_restored_image_from_motion_blur = inverse_filter(motion_blurred_image, kernel_motion_blur)
cv2.imshow('Inverse Restored Image from Motion Blur', inverse_restored_image_from_motion_blur)
cv2.waitKey()

-1

In [239]:
def inverse_filter(image, psf):
    inversed = np.zeros_like(image)
    for i in range(3):
        inversed[:,:,i] = div(image[:,:,i],psf)
    return inversed.astype(np.uint8)

inverse_restored_image_from_motion_blur = inverse_filter(motion_blurred_image, kernel_motion_blur)
cv2.imshow('Inverse Restored Image from Motion Blur', inverse_restored_image_from_motion_blur)
cv2.waitKey()

C:\Users\10037\AppData\Local\Temp\ipykernel_22988\1595068668.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  inversed[:,:,i] = div(image[:,:,i],psf)


-1

In [92]:
def wiener_filter(image, psf, k=0.01):
    for i in range(3):
        input_fft = np.fft.fft2(image[:,:,i])
        psf_fft = np.fft.fft2(psf[:,:,i]) + 1e-8
        psf_fft_conj = np.conj(psf_fft)
        result = np.fft.ifft2((psf_fft_conj) / (psf_fft * psf_fft_conj + k) * input_fft)
        result = np.abs(result)
        image[:,:,i] = result / result.max() * 255
    return image.astype(np.uint8)

In [94]:
motion_psf = np.zeros_like(motion_blurred_image)
motion_psf[:, int(motion_psf.shape[1]/2-motion_kernel_size/2):int(motion_psf.shape[1]/2+motion_kernel_size/2)] = 1/motion_kernel_size
inverse_restored_image_from_motion_blur = inverse_filter(motion_blurred_image, motion_psf)
wiener_restored_image_from_motion_blur = wiener_filter(motion_blurred_image, motion_psf)

In [100]:
defocus_psf0 = cv2.getGaussianKernel(defocus_kernel_size*4+1, defocus_kernel_size) * cv2.getGaussianKernel(defocus_kernel_size*4+1, defocus_kernel_size).T
defocus_psf = np.zeros((defocus_psf0.shape[0],defocus_psf0.shape[1],3))
defocus_psf[:,:,1] = defocus_psf[:,:,2] = defocus_psf0
inverse_restored_image_from_defocus_blur = inverse_filter(defocus_blurred_image, defocus_psf)
wiener_restored_image_from_defocus_blur = wiener_filter(defocus_blurred_image, defocus_psf)

ValueError: operands could not be broadcast together with shapes (900,1600) (61,61) 

In [101]:
cv2.imshow('Original Image', image)
cv2.imshow('Motion Blurred Image', motion_blurred_image)
cv2.imshow('Defocus Blurred Image', defocus_blurred_image)
cv2.imshow('Inverse Restored Image from Motion Blur', inverse_restored_image_from_motion_blur)
#cv2.imshow('Wiener Restored Image from Motion Blur', wiener_restored_image_from_motion_blur)
#cv2.imshow('Inverse Restored Image from Defocus Blur', inverse_restored_image_from_defocus_blur)
#cv2.imshow('Wiener Restored Image from Defocus Blur', wiener_restored_image_from_defocus_blur)

cv2.waitKey(0)

-1